In [1]:
#search for import above the current directory
import sys
import os
from pathlib import Path
cwd = Path().resolve()

sys.path.insert(1, os.path.join(cwd, 'Read Process CSV'))

import numpy as np
import pandas as pd
from tqdm import tqdm
import json
from scipy import stats

import dataAnalyzer_FTStat
import dataScreen_FTStat
import orbiStandardize

#Set up the inputs

#Folder with FT Statistic-ified files. All the files need to be processed using the same metrics.
folderPath = "Orbi_120k"
SmpStd = ['Std','A'] * 3 + ['Std','F'] * 3 + ['Std'] + ['Std','K'] * 3 + ['Std','I'] * 3 + ['Std'] + ['Std','M'] * 3 + ['Std'] + ['Std','C'] * 3 + ['Std'] + ['Std','THR2'] * 3 + ['Std','ARC'] * 3 + ['Std'] + ['Std','BTC'] * 3 + ['Std','THR1'] * 3 + ['Std']
Replicate = ['1','1','2','2','3','3','4','1','1','2','2','3','3'] * 2 + ['1','1','2','2','3','3','4'] * 2

fragmentDict = {'96':['18O','D','17O','13C','Unsub']}

fragmentMostAbundant = ['Unsub']

massStr = []
fragmentIsotopeList = []
for i, v in fragmentDict.items():
    massStr.append(i)
    fragmentIsotopeList.append(v)
    
cullByTime = True
cullTimes = [(3,40)]

#Any specific properties you want to cull on
cullOn = "TIC*IT"
#Multiple of SD you want to cull beyond for the cullOn property
cull_amount = 3
#Whether you want to cull zero scans
cullZeroScans = False
#Whether you want to calculate weighted averages based on NL height (specifically designed for GC elution but widely applicable!)
weightByNLHeight = False
#Whether you want to output each file as you process it, and where you want it to go:
fileOutputPath = None

percentAbundance = False

extractedData120k = {}

rtnAllFilesDF, mergedDict, allOutputDict = dataAnalyzer_FTStat.calc_Folder_Output(folderPath, cullOn=cullOn, cullAmount=cull_amount,
                                               cullByTime=cullByTime, cullTimes = cullTimes, 
                                               fragmentIsotopeList = fragmentIsotopeList, 
                                               fragmentMostAbundant = fragmentMostAbundant, debug = True, fileExt = '.txt', 
                                               massStrList = list(fragmentDict.keys()),
                                               Microscans = 10)

failedRSE = dataScreen_FTStat.RSESNScreen(allOutputDict)
failedZero = dataScreen_FTStat.zeroCountsScreen(mergedDict, fragmentDict['96'], threshold = 0)
failedInternal = dataScreen_FTStat.internalStabilityScreenSubsequence(mergedDict, fragmentDict['96'], 'Unsub', priorSubsequenceLength = 100, testSubsequenceLength = 100, thresholdConstant = 0.64)
failedTICIT = dataScreen_FTStat.TICITScreen(mergedDict)

fileKeys = list(mergedDict.keys())
for subKey, subData in failedRSE.items():
    failedRSE[subKey] = [fileKeys[x] for x in subData]

failedAny = {}
for subKey, subData in failedRSE.items():
    failedAny[subKey + '/Unsub'] = set(subData) | set(failedZero[subKey]) | set(failedZero['Unsub']) | set(failedInternal[subKey]) | set(failedInternal['Unsub']) | set(failedTICIT)

extractedData120k = dataAnalyzer_FTStat.folderOutputToDict(rtnAllFilesDF)

20211207_10_MPA_STD_01.txt
20211207_11_MPA_A_01.txt
20211207_12_MPA_STD_02.txt
20211207_13_MPA_A_02.txt
20211207_14_MPA_STD_03.txt
20211207_15_MPA_A_03.txt
20211207_16_MPA_STD_04.txt
20211207_17_MPA_F_01.txt
20211207_18_MPA_STD_05.txt
20211207_19_MPA_F_02.txt
20211207_20_MPA_STD_06.txt
20211207_21_MPA_F_03.txt
20211207_22_MPA_STD_07.txt
20211208_11_MPA_STD_01.txt
20211208_12_MPA_K_01.txt
20211208_13_MPA_STD_02.txt
20211208_14_MPA_K_02.txt
20211208_15_MPA_STD_03.txt
20211208_16_MPA_K_03.txt
20211208_17_MPA_STD_04.txt
20211208_18_MPA_I_01.txt
20211208_19_MPA_STD_05.txt
20211208_20_MPA_I_02.txt
20211208_21_MPA_STD_06.txt
20211208_22_MPA_I_03.txt
20211208_23_MPA_STD_04.txt
20211209_05_MPA_STD_01.txt
20211209_06_MPA_M_01.txt
20211209_07_MPA_STD_02.txt
20211209_08_MPA_M_02.txt
20211209_09_MPA_STD_03.txt
20211209_10_MPA_M_03.txt
20211209_11_MPA_STD_04.txt
20211210_09_MPA_STD_01.txt
20211210_10_MPA_C_01.txt
20211210_11_MPA_STD_01_Repeat.txt
20211210_12_MPA_C_02.txt
20211210_13_MPA_STD_03.txt
2

In [2]:
failedAny

{'18O/Unsub': {'20211207_12_MPA_STD_02.txt',
  '20211207_13_MPA_A_02.txt',
  '20211207_22_MPA_STD_07.txt',
  '20211208_13_MPA_STD_02.txt',
  '20211208_19_MPA_STD_05.txt',
  '20211210_09_MPA_STD_01.txt',
  '20211210_10_MPA_C_01.txt',
  '20211210_11_MPA_STD_01_Repeat.txt',
  '20230502_04_MPA_SIG_STD_01.txt',
  '20230502_05_MPA_THR2_SMP_02.txt',
  '20230502_07_MPA_THR2_SMP_03.txt',
  '20230502_09_MPA_ARC_SMP_01.txt',
  '20230502_10_MPA_SIG_STD_02.txt',
  '20230502_12_MPA_SIG_STD_03.txt'},
 'D/Unsub': {'20211208_23_MPA_STD_04.txt',
  '20230502_05_MPA_THR2_SMP_02.txt',
  '20230502_07_MPA_THR2_SMP_03.txt',
  '20230502_09_MPA_ARC_SMP_01.txt',
  '20230502_10_MPA_SIG_STD_02.txt'},
 '17O/Unsub': {'20211207_18_MPA_STD_05.txt',
  '20211207_21_MPA_F_03.txt',
  '20211208_15_MPA_STD_03.txt',
  '20211209_06_MPA_M_01.txt',
  '20230502_05_MPA_THR2_SMP_02.txt',
  '20230502_07_MPA_THR2_SMP_03.txt',
  '20230502_09_MPA_ARC_SMP_01.txt',
  '20230502_10_MPA_SIG_STD_02.txt'},
 '13C/Unsub': {'20230502_05_MPA_THR

In [3]:
checkThresholdConstant = False
if checkThresholdConstant:
    maxes = []
    for i in tqdm(range(1000)):
        array = np.random.normal(size = 4570)

        # Reshape the array into a 2D array with 10 columns
        num_rows = 4570 // 10
        reshaped_array = array[:num_rows * 10].reshape(num_rows, 10)

        # Calculate the average along axis 1 (columns)
        averages = np.mean(reshaped_array, axis=1)

        outlier = dataScreen_FTStat.subsequenceOutlierDetection(pd.Series(averages), priorSubsequenceLength = 100, testSubsequenceLength = 100)
        maxes.append(outlier.max())

    np.array(maxes).mean() + 4 * np.array(maxes).std()

In [4]:
checkTIC = False
if checkTIC:
    for thisSmp in ['A','F','K','I','M','C','ARC','BTC','THR1','THR2']:
        firstIdx = SmpStd.index(thisSmp)
        thisSmpIndices = [firstIdx, firstIdx + 2, firstIdx + 4]
        thisStdIndices = [firstIdx-1, firstIdx +1, firstIdx + 3, firstIdx +5]

        allSmpTic = []
        for smpIdx in thisSmpIndices:
            smpName = list(mergedDict.keys())[smpIdx]
            thisDf = mergedDict[smpName][0]
            allSmpTic += list(thisDf['tic'])

        allStdTic = []
        for stdIdx in thisStdIndices:
            stdName = list(mergedDict.keys())[stdIdx]
            thisDf = mergedDict[stdName][0]
            allStdTic += list(thisDf['tic'])

        smpTic = np.array(allSmpTic)
        stdTic = np.array(allStdTic)
        print(thisSmp + ' ' + f'{smpTic.mean():.2f}' + ' ' + f'{smpTic.std() / smpTic.mean():.2f}')
        print( f'{stdTic.mean():.2f}' + ' ' + f'{stdTic.std() / stdTic.mean():.2f}')

In [5]:
byIndex = {}
bySample = {}
for thisSmp in ['A','F','K','I','M','C','ARC','BTC','THR1','THR2']:
    bySample[thisSmp] = {}
    firstIdx = SmpStd.index(thisSmp)
    thisIndices = list(range(firstIdx -1, firstIdx + 6))
    byIndex[thisSmp] = thisIndices

    for ratioKey in ['13C/Unsub','D/Unsub']:
        valDict = orbiStandardize.extractThisRun(extractedData120k, failedAny, thisIndices, ratioKey)
        vals, errs = orbiStandardize.standardizeOneRun(valDict['Avg'], valDict['RelStdError'], standardize = 'linear')
        deltas = 1000*(vals-1)

        bySample[thisSmp][ratioKey] = {}
        bySample[thisSmp][ratioKey]['Ratios'] = list(vals)
        bySample[thisSmp][ratioKey]['RSEs'] = list(errs)
        bySample[thisSmp][ratioKey]['Deltas'] = list(deltas)
        bySample[thisSmp][ratioKey]['DeltasErr'] = list(1000*errs)
        bySample[thisSmp][ratioKey]['Delta Mean'] = deltas.mean()
        bySample[thisSmp][ratioKey]['ER'] = deltas.std()

with open('120k_Data.json', 'w', encoding='utf-8') as f:
    json.dump(bySample, f, ensure_ascii=False, indent=4)